In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Imports OK")

## 1. Chargement des Données

Source: API locale ou données CSV

In [ ]:
# Tentative de connexion à l'API locale
API_BASE_URL = "http://localhost:8000/api/v1"

try:
    # Test de connexion
    response = requests.get(f"{API_BASE_URL}/health", timeout=5)
    if response.status_code == 200:
        print("[OK] API accessible")
        USE_API = True
    else:
        print("[ATTENTION] API non accessible, fallback sur CSV")
        USE_API = False
except:
    print("[ATTENTION] API non disponible, utilisation des CSV")
    USE_API = False

# Chargement des données usagers (avec âge)
if USE_API:
    # Via API
    try:
        response = requests.get(f"{API_BASE_URL}/stats/usagers?limit=10000")
        df_usagers = pd.DataFrame(response.json())
        print(f"Données: {len(df_usagers)} lignes chargées via API")
    except Exception as e:
        print(f"[ERREUR] Erreur API: {e}")
        df_usagers = pd.DataFrame()
else:
    # Via CSV
    try:
        df_usagers = pd.read_csv('../data/clean/clean_usagers.csv')
        print(f"Données: {len(df_usagers)} lignes chargées depuis CSV")
    except FileNotFoundError:
        print("[ATTENTION] Fichier CSV non trouvé")
        print("Conseil: Exécutez d'abord le pipeline ETL: python src/pipeline/run_pipeline.py")
        df_usagers = pd.DataFrame()

if not df_usagers.empty:
    print(f"\nColonnes disponibles: {list(df_usagers.columns)}")
    print(f"\nAperçu des données:")
    display(df_usagers.head())

## 2. Données de Référence INSEE/ONISR

**Sources**:
- Population par âge: INSEE (données publiques)
- Taux de détention du permis: Enquêtes ONISR
- Kilomètres parcourus: Estimations académiques

In [ ]:
# Données de référence (estimations 2023-2024)
# Source: INSEE + ONISR + Enquêtes mobilité

population_reference = {
    'tranche_age': ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75+'],
    'population': [5_200_000, 8_100_000, 8_400_000, 8_800_000, 8_200_000, 6_000_000, 4_500_000],
    'taux_permis': [0.72, 0.88, 0.90, 0.90, 0.88, 0.85, 0.78],  # % ayant le permis
    'km_annuels_moyen': [8_000, 13_000, 15_000, 14_000, 12_000, 9_000, 6_000],  # km/an
    'freq_utilisation': [0.65, 0.85, 0.90, 0.90, 0.85, 0.75, 0.60]  # % conduisant régulièrement
}

df_ref = pd.DataFrame(population_reference)

# Calcul des conducteurs actifs
df_ref['conducteurs_avec_permis'] = (df_ref['population'] * df_ref['taux_permis']).astype(int)
df_ref['conducteurs_actifs'] = (df_ref['conducteurs_avec_permis'] * df_ref['freq_utilisation']).astype(int)
df_ref['km_totaux'] = (df_ref['conducteurs_actifs'] * df_ref['km_annuels_moyen']).astype(int)

print("Données de référence (estimations):\n")
display(df_ref)

print(f"\nRésumé:")
print(f"  - Population totale 18+: {df_ref['population'].sum():,}")
print(f"  - Conducteurs avec permis: {df_ref['conducteurs_avec_permis'].sum():,}")
print(f"  - Conducteurs actifs: {df_ref['conducteurs_actifs'].sum():,}")
print(f"  - Km totaux annuels: {df_ref['km_totaux'].sum():,}")

## 3. Préparation des Données d'Accidents

Agrégation par tranche d'âge et calcul des métriques

In [ ]:
if not df_usagers.empty:
    # Détection des colonnes disponibles
    age_col = 'age' if 'age' in df_usagers.columns else None
    gravite_col = 'gravite' if 'gravite' in df_usagers.columns else 'grav' if 'grav' in df_usagers.columns else None
    
    if age_col and gravite_col:
        # Nettoyage
        df_work = df_usagers[[age_col, gravite_col]].copy()
        df_work = df_work[df_work[age_col] > 0]  # Supprimer âges invalides
        df_work = df_work[df_work[age_col] >= 18]  # Seulement 18+
        df_work = df_work[df_work[age_col] <= 100]  # Âges réalistes
        
        # Créer tranches d'âge
        bins = [18, 25, 35, 45, 55, 65, 75, 100]
        labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75+']
        df_work['tranche_age'] = pd.cut(df_work[age_col], bins=bins, labels=labels, right=False)
        
        # Agrégation
        df_accidents_age = df_work.groupby('tranche_age').agg({
            age_col: 'count',
            gravite_col: ['mean', lambda x: (x >= 3).sum(), lambda x: (x == 4).sum()]
        }).reset_index()
        
        df_accidents_age.columns = ['tranche_age', 'nombre_usagers', 'gravite_moyenne', 'graves', 'deces']
        
        print("Accidents par tranche d'âge (DONNÉES BRUTES):\n")
        display(df_accidents_age)
        
    else:
        print(f"[ERREUR] Colonnes manquantes: age={age_col}, gravite={gravite_col}")
        df_accidents_age = None
else:
    print("[ATTENTION] Pas de données usagers chargées")
    df_accidents_age = None

## 4. Normalisation par Exposition

**Calcul des taux normalisés**:
- Accidents pour 100 000 conducteurs actifs
- Accidents pour 100 millions de km parcourus

In [ ]:
if df_accidents_age is not None:
    # Fusion avec données de référence
    df_analyse = df_accidents_age.merge(df_ref, on='tranche_age', how='left')
    
    # Calcul des taux normalisés
    df_analyse['taux_pour_100k_conducteurs'] = (
        df_analyse['nombre_usagers'] / df_analyse['conducteurs_actifs'] * 100_000
    ).round(2)
    
    df_analyse['taux_pour_100M_km'] = (
        df_analyse['nombre_usagers'] / df_analyse['km_totaux'] * 100_000_000
    ).round(2)
    
    df_analyse['taux_graves_pour_100k'] = (
        df_analyse['graves'] / df_analyse['conducteurs_actifs'] * 100_000
    ).round(2)
    
    df_analyse['taux_deces_pour_100k'] = (
        df_analyse['deces'] / df_analyse['conducteurs_actifs'] * 100_000
    ).round(2)
    
    print("RÉSULTATS NORMALISÉS:\n")
    display(df_analyse[[
        'tranche_age', 'nombre_usagers', 'conducteurs_actifs',
        'taux_pour_100k_conducteurs', 'taux_pour_100M_km',
        'gravite_moyenne', 'taux_graves_pour_100k', 'taux_deces_pour_100k'
    ]])
    
    # Calcul des risques relatifs (référence: 35-44 ans)
    ref_age = '35-44'
    taux_ref = df_analyse[df_analyse['tranche_age'] == ref_age]['taux_pour_100k_conducteurs'].values[0]
    
    df_analyse['risque_relatif'] = (df_analyse['taux_pour_100k_conducteurs'] / taux_ref).round(2)
    
    print(f"\nRISQUES RELATIFS (ref: {ref_age} = 1.00):\n")
    for _, row in df_analyse.iterrows():
        marker = "[ÉLEVÉ]" if row['risque_relatif'] > 1.5 else "[MODÉRÉ]" if row['risque_relatif'] > 1.2 else "[FAIBLE]"
        print(f"{marker} {row['tranche_age']:8} : {row['risque_relatif']:.2f}x")
else:
    print("[ATTENTION] Analyse impossible sans données")

## 5. Visualisations Comparatives

### A. Données Brutes vs Normalisées

In [ ]:
if df_accidents_age is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graphique 1: Nombre d'accidents bruts
    axes[0].bar(df_analyse['tranche_age'], df_analyse['nombre_usagers'], color='steelblue', alpha=0.7)
    axes[0].set_title('[TROMPEUR] DONNÉES BRUTES\nNombre d\'usagers impliqués par âge', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Tranche d\'âge', fontsize=12)
    axes[0].set_ylabel('Nombre d\'usagers', fontsize=12)
    axes[0].grid(axis='y', alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Graphique 2: Taux normalisé
    colors = ['red' if x > 1.5 else 'orange' if x > 1.2 else 'green' for x in df_analyse['risque_relatif']]
    axes[1].bar(df_analyse['tranche_age'], df_analyse['taux_pour_100k_conducteurs'], color=colors, alpha=0.7)
    axes[1].set_title('[RÉALITÉ] DONNÉES NORMALISÉES\nTaux pour 100 000 conducteurs actifs', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Tranche d\'âge', fontsize=12)
    axes[1].set_ylabel('Taux d\'accidents pour 100k', fontsize=12)
    axes[1].grid(axis='y', alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("\nINTERPRÉTATION:")
    print("  - Graphique GAUCHE: Montre moins d'accidents jeunes (TROMPEUR car ils conduisent moins)")
    print("  - Graphique DROITE: Montre le VRAI risque par exposition (jeunes = risque élevé)")

### B. Risque Relatif et Gravité

In [ ]:
if df_accidents_age is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graphique 1: Risque relatif
    axes[0].plot(df_analyse['tranche_age'], df_analyse['risque_relatif'], 
                marker='o', linewidth=3, markersize=10, color='darkred')
    axes[0].axhline(y=1, color='green', linestyle='--', linewidth=2, label='Risque de référence (35-44 ans)')
    axes[0].fill_between(range(len(df_analyse)), 0, df_analyse['risque_relatif'], alpha=0.2, color='red')
    axes[0].set_title('Risque Relatif par Âge\n(Référence: 35-44 ans = 1.0)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Tranche d\'âge', fontsize=12)
    axes[0].set_ylabel('Risque relatif', fontsize=12)
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)
    
    # Graphique 2: Gravité moyenne
    axes[1].bar(df_analyse['tranche_age'], df_analyse['gravite_moyenne'], color='coral', alpha=0.7)
    axes[1].axhline(y=df_analyse['gravite_moyenne'].mean(), color='blue', 
                   linestyle='--', linewidth=2, label=f'Moyenne globale: {df_analyse["gravite_moyenne"].mean():.2f}')
    axes[1].set_title('Gravité Moyenne des Accidents\n(1=Indemne, 2=Léger, 3=Grave, 4=Mortel)', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Tranche d\'âge', fontsize=12)
    axes[1].set_ylabel('Gravité moyenne', fontsize=12)
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].set_ylim([1, 4])
    
    plt.tight_layout()
    plt.show()

### C. Taux d'Accidents Graves et Décès

In [ ]:
if df_accidents_age is not None:
    fig, ax = plt.subplots(figsize=(14, 7))
    
    x = np.arange(len(df_analyse))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, df_analyse['taux_graves_pour_100k'], width, 
                   label='Accidents graves (hosp.)', color='orange', alpha=0.8)
    bars2 = ax.bar(x + width/2, df_analyse['taux_deces_pour_100k'], width, 
                   label='Décès', color='darkred', alpha=0.8)
    
    ax.set_title('Taux d\'Accidents Graves et Décès\n(pour 100 000 conducteurs actifs)', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Tranche d\'âge', fontsize=13)
    ax.set_ylabel('Taux pour 100 000 conducteurs', fontsize=13)
    ax.set_xticks(x)
    ax.set_xticklabels(df_analyse['tranche_age'], rotation=45)
    ax.legend(fontsize=12)
    ax.grid(axis='y', alpha=0.3)
    
    # Annotations
    for i, (grave, deces) in enumerate(zip(df_analyse['taux_graves_pour_100k'], df_analyse['taux_deces_pour_100k'])):
        ax.text(i - width/2, grave + 1, f'{grave:.1f}', ha='center', fontsize=9)
        ax.text(i + width/2, deces + 0.5, f'{deces:.1f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 6. Analyse Statistique

### Test de significativité des différences

In [ ]:
if df_accidents_age is not None:
    from scipy import stats
    
    print("TESTS STATISTIQUES\n")
    print("=" * 70)
    
    # Test ANOVA: différence significative entre tranches d'âge?
    taux_par_age = df_analyse['taux_pour_100k_conducteurs'].values
    
    print("\n1. Variance des taux d'accidents par âge:")
    print(f"   - Écart-type: {taux_par_age.std():.2f}")
    print(f"   - Coefficient de variation: {(taux_par_age.std() / taux_par_age.mean() * 100):.1f}%")
    print(f"   - Min: {taux_par_age.min():.2f} | Max: {taux_par_age.max():.2f}")
    print(f"   - Ratio Max/Min: {taux_par_age.max() / taux_par_age.min():.2f}x")
    
    # Comparaison jeunes (18-24) vs expérimentés (35-44)
    print("\n2. Comparaison Jeunes (18-24) vs Expérimentés (35-44):")
    jeunes = df_analyse[df_analyse['tranche_age'] == '18-24']
    exp = df_analyse[df_analyse['tranche_age'] == '35-44']
    
    if not jeunes.empty and not exp.empty:
        taux_jeunes = jeunes['taux_pour_100k_conducteurs'].values[0]
        taux_exp = exp['taux_pour_100k_conducteurs'].values[0]
        diff_pct = ((taux_jeunes - taux_exp) / taux_exp * 100)
        
        print(f"   - Taux jeunes: {taux_jeunes:.2f} pour 100k")
        print(f"   - Taux expérimentés: {taux_exp:.2f} pour 100k")
        print(f"   - Différence: {diff_pct:+.1f}%")
        print(f"   - Facteur multiplicatif: {taux_jeunes/taux_exp:.2f}x")
        
        if diff_pct > 20:
            print("   [OK] CONCLUSION: Les jeunes conducteurs ont un risque SIGNIFICATIVEMENT plus élevé")
        else:
            print("   [ATTENTION] CONCLUSION: Différence modérée")
    
    # Gravité
    print("\n3. Analyse de la gravité:")
    gravite_jeunes = jeunes['gravite_moyenne'].values[0] if not jeunes.empty else 0
    gravite_exp = exp['gravite_moyenne'].values[0] if not exp.empty else 0
    
    print(f"   - Gravité moyenne jeunes: {gravite_jeunes:.2f}/4")
    print(f"   - Gravité moyenne expérimentés: {gravite_exp:.2f}/4")
    if gravite_jeunes > 0 and gravite_exp > 0:
        print(f"   - Différence: {((gravite_jeunes - gravite_exp) / gravite_exp * 100):+.1f}%")

## 7. Conclusions et Recommandations

### Synthèse des Résultats

In [ ]:
if df_accidents_age is not None:
    print("="*80)
    print("RAPPORT DE SYNTHÈSE: RISQUE PAR ÂGE ET EXPOSITION")
    print("="*80)
    
    print("\nFINDINGS PRINCIPAUX:\n")
    
    # Finding 1: Biais des données brutes
    print("1. BIAIS D'EXPOSITION CONFIRMÉ")
    jeunes_brut = df_analyse[df_analyse['tranche_age'] == '18-24']['nombre_usagers'].values[0]
    total_brut = df_analyse['nombre_usagers'].sum()
    print(f"   - Données brutes: {jeunes_brut:,} accidents jeunes ({jeunes_brut/total_brut*100:.1f}% du total)")
    print(f"   - Semblent 'moins dangereux' car ils conduisent MOINS")
    print(f"   [FAUX] Interprétation brute = FAUSSE")
    
    # Finding 2: Après normalisation
    print("\n2. APRÈS NORMALISATION PAR EXPOSITION")
    risque_jeunes = df_analyse[df_analyse['tranche_age'] == '18-24']['risque_relatif'].values[0]
    print(f"   - Risque relatif jeunes (18-24): {risque_jeunes:.2f}x (ref: 35-44)")
    print(f"   - Taux accidents jeunes: {df_analyse[df_analyse['tranche_age'] == '18-24']['taux_pour_100k_conducteurs'].values[0]:.1f} pour 100k")
    print(f"   [OK] Les jeunes conducteurs sont PLUS à risque (quand on normalise)")
    
    # Finding 3: Tranches à risque
    print("\n3. TRANCHES D'ÂGE À RISQUE ÉLEVÉ")
    risque_eleve = df_analyse[df_analyse['risque_relatif'] > 1.3].sort_values('risque_relatif', ascending=False)
    for _, row in risque_eleve.iterrows():
        print(f"   [ÉLEVÉ] {row['tranche_age']:8} : {row['risque_relatif']:.2f}x | Gravité: {row['gravite_moyenne']:.2f}/4")
    
    # Finding 4: Gravité
    print("\n4. ANALYSE DE LA GRAVITÉ")
    gravite_max_age = df_analyse.loc[df_analyse['gravite_moyenne'].idxmax(), 'tranche_age']
    gravite_max_val = df_analyse['gravite_moyenne'].max()
    print(f"   - Tranche avec gravité maximale: {gravite_max_age} ({gravite_max_val:.2f}/4)")
    print(f"   - Gravité moyenne globale: {df_analyse['gravite_moyenne'].mean():.2f}/4")
    
    print("\n" + "="*80)
    print("RECOMMANDATIONS:\n")
    print("1. POUR L'ASSURANCE:")
    print("   - Appliquer surprime jeunes conducteurs (18-24): +50% à +100%")
    print("   - Bonus progressif après 3-5 ans sans sinistre")
    print("   - Programmes de prévention ciblés")
    
    print("\n2. POUR LA PRÉVENTION:")
    print("   - Campagnes spécifiques jeunes (nuit, alcool, vitesse)")
    print("   - Formation post-permis obligatoire")
    print("   - Sensibilisation seniors (65+) sur fragilité")
    
    print("\n3. POUR LES ANALYSES FUTURES:")
    print("   - TOUJOURS normaliser par exposition (conducteurs actifs ou km)")
    print("   - Ne JAMAIS comparer les nombres bruts entre groupes d'âge")
    print("   - Analyser séparément fréquence ET gravité")
    
    print("\n" + "="*80)
    print("[OK] ANALYSE TERMINÉE")
    print("="*80)
else:
    print("[ATTENTION] Analyse impossible: données non disponibles")

## 8. Export des Résultats

In [ ]:
if df_accidents_age is not None:
    # Sauvegarder les résultats
    output_file = '../data/analysis/risk_normalization_results.csv'
    
    try:
        import os
        os.makedirs('../data/analysis', exist_ok=True)
        df_analyse.to_csv(output_file, index=False)
        print(f"[OK] Résultats exportés: {output_file}")
    except Exception as e:
        print(f"[ATTENTION] Impossible d'exporter: {e}")
    
    print("\nColonnes exportées:")
    print(list(df_analyse.columns))

---

## Notes Méthodologiques

### Sources des Estimations

1. **Population par âge**: INSEE, Bilan démographique 2023
2. **Taux de détention du permis**: 
   - 18-24 ans: ~72% (source: Enquête Mobilité INSEE)
   - 25-64 ans: ~88-90%
   - 65+ ans: ~75-85% (déclin après 75 ans)

3. **Kilomètres annuels**: 
   - Enquête ENTD (Enquête Nationale Transports et Déplacements)
   - Moyenne France: 12 000 km/an/conducteur
   - Jeunes: 8 000 km (usage occasionnel)
   - Actifs: 13-15 000 km (trajets domicile-travail)
   - Retraités: 6-9 000 km

4. **Fréquence d'utilisation**:
   - % de détenteurs du permis qui conduisent régulièrement
   - Jeunes: 65% (covoiturage, transports en commun)
   - Actifs: 85-90%
   - Seniors: 60-75% (mobilité réduite après 75 ans)

### Limites de l'Analyse

- [ATTENTION] **Estimations**: Les données d'exposition sont des estimations, pas des mesures exactes
- [ATTENTION] **Agrégation**: Les tranches d'âge masquent la variabilité intra-groupe
- [ATTENTION] **Biais de survie**: Les accidents mortels des jeunes peuvent être sous-représentés
- [ATTENTION] **Facteurs confondants**: Genre, type de véhicule, zone géographique non contrôlés

### Pour Aller Plus Loin

1. Croiser avec données ONISR (taux officiels)
2. Segmenter par genre (hommes jeunes = risque max)
3. Analyser par type de trajet (loisirs vs travail)
4. Modèle de régression multivarié (âge + expérience + genre + type véhicule)

---

**Notebook créé le**: 30 janvier 2026  
**Projet**: Accidents Routiers - Analyse & API  
**Contact**: GitHub @Gouesse05